In [67]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

In [68]:
# Print TensorFlow and Pillow versions for confirmation
print("TensorFlow Version:", tf.__version__)
print("Pillow Version:", Image.__version__)

TensorFlow Version: 2.17.0
Pillow Version: 10.4.0


In [69]:
# Clear any previous session
tf.keras.backend.clear_session()

In [80]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf

def load_images_from_directory(directory, target_size):
    images = []
    labels = []
    class_names = sorted(os.listdir(directory))
    for class_index, class_name in enumerate(class_names):
        class_dir = os.path.join(directory, class_name)
        if os.path.isdir(class_dir):
            for file_name in os.listdir(class_dir):
                file_path = os.path.join(class_dir, file_name)
                try:
                    img = Image.open(file_path).resize(target_size)
                    img_array = np.array(img) / 255.0  # Normalize pixel values
                    images.append(img_array)
                    labels.append(class_index)
                except Exception as e:
                    print(f"Error loading image {file_path}: {e}")
    return np.array(images), np.array(labels), class_names

# Load images from pre-split directories
train_dir = 'images/meta/train'
validation_dir = 'images/meta/validation'
test_dir = 'images/meta/test'
target_size = (224, 224)

train_images, train_labels, class_names = load_images_from_directory(train_dir, target_size)
validation_images, validation_labels, _ = load_images_from_directory(validation_dir, target_size)
test_images, test_labels, _ = load_images_from_directory(test_dir, target_size)

# Convert labels to categorical
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=len(class_names))
validation_labels = tf.keras.utils.to_categorical(validation_labels, num_classes=len(class_names))
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=len(class_names))


In [81]:
# Parameters
n_classes = 30  # Number of categories
batch_size = 32
width, height = 224, 224

In [82]:
# Data generators with data augmentation
train_data_gen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest'
)

validation_data_gen = ImageDataGenerator(rescale=1. / 255)
test_data_gen = ImageDataGenerator(rescale=1. / 255)

train_gen = train_data_gen.flow_from_directory(
    train_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_gen = validation_data_gen.flow_from_directory(
    validation_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_gen = test_data_gen.flow_from_directory(
    test_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 6791 images belonging to 30 classes.
Found 2072 images belonging to 30 classes.
Found 2102 images belonging to 30 classes.


In [83]:
# Model definition using InceptionV3
inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(height, width, 3))
layer = inception.output
layer = GlobalAveragePooling2D()(layer)
layer = Dense(128, activation='relu')(layer)
layer = Dropout(0.2)(layer)
predictions = Dense(n_classes, activation='softmax')(layer)

model = Model(inputs=inception.input, outputs=predictions)

In [84]:
# Compile the model
model.compile(
    optimizer=SGD(learning_rate=0.0001, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [85]:
# Callbacks
checkpointer = ModelCheckpoint(filepath='best_recyclable_model.keras', save_best_only=True)
csv_logger = CSVLogger('recyclable_model_history.log')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

In [86]:
# Train the model
history = model.fit(
    train_gen,
    steps_per_epoch=train_gen.samples // batch_size,
    validation_data=validation_gen,
    validation_steps=validation_gen.samples // batch_size,
    epochs=30,
    callbacks=[csv_logger, checkpointer, early_stopping, reduce_lr]
)

# Save the final model
model.save('recyclable_model_final.keras')

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

In [ ]:
# Plot training & validation accuracy and loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

In [62]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from PIL import Image

print("TensorFlow Version:", tf.__version__)
print("Pillow Version:", Image.__version__)

# Test loading an image
try:
    img = load_img('"D:\Recyclable_image_processing\images\images\glass_cosmetic_containers\default\Image_3.png"')  # Provide a valid path to an image
    print("Image loaded successfully!")
except Exception as e:
    print("Error loading image:", e)


TensorFlow Version: 2.17.0
Pillow Version: 10.4.0
Error loading image: Could not import PIL.Image. The use of `load_img` requires PIL.
